# Linear Solver Performance: XDG Stokes, Single Core
### Part 1, Benchmark Setup and Execution

In [1]:
Console.WriteLine("Execution Date/time is " + DateTime.Now);

The below script needs to be able to find the current output cell; this is an easy method to get it.

Execution Date/time is 11/12/2021 5:24:25 AM


In [2]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSFE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [3]:
var myBatch = GetDefaultQueue();
myBatch

DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,AllowedDatabasesPaths
\\fdygitrunner\ValidationTests,True,MSHPC-AllNodes,dotnet,FDY\jenkinsci,DC2,"[ hpccluster, hpccluster2, fdygitrunner ]",Normal,True,[ \\fdygitrunner\ValidationTests == ]


In [4]:
wmg.Init("LinslvPerf_XdgStokes");
wmg.AllJobs

Project name is set to 'LinslvPerf_XdgStokes'.
Opening existing database '\\fdygitrunner\ValidationTests\LinslvPerf_XdgStokes'.


## Utility definitions

In [5]:
static class Utils {
    // DOF per cell in 3D for one variable
    public static int Np(int p) {
        return (p*p*p + 6*p*p + 11*p + 6)/6;
    }    
    
    /*
    //Non-equidistant nodes
    public static double[] SinLinSpacing(double l, double r, double a, int n) {
        double[] linnodes = GenericBlas.Linspace(-Math.PI * 0.5, Math.PI * 0.5, n);
        double[] linnodes2 = GenericBlas.Linspace(-1, 1, n);
        double[] nodes = new double[n];

        for (int i = 0; i < n; i++)
            //nodes[i] = linnodes2[i] * (1 - a) + (1.0 - Math.Sin(linnodes[i])) * a;
            nodes[i] = linnodes2[i] * (1 - a) + Math.Sin(linnodes[i])*a;

        for (int i = 0; i < n; i++)
            nodes[i] = nodes[i] * (r - l)*0.5 + l;
        return nodes;
    }
    */
}

## Init grids and save to database

Note: The dimension of the domain $(-1,1)^3$ are assumed to be **centimeters**!
In this benchmark, realistic physical values are used, i.e. densities and viscosities of 
- water for Phase A (inside the droplet)
- air for Phase B (surounding)
The droplet is assumed to be in the millimeter range; therefore, all lenght-, area-, and volume-related properties 
must also be set in centimeters, e.g. the density of water is $10^{-3} \text{kg}/\text{cm}^3$.

In [6]:
int[] Resolutions_3D = new int[] { 8, 16, 24, 32, 48, 64 };
//int[] Resolutions_3D = new int[] { 2 };
IGridInfo[] grids = new IGridInfo[Resolutions_3D.Length];
for(int cnt = 0; cnt < Resolutions_3D.Length; cnt++) {
    int Res = Resolutions_3D[cnt];    
    
    double[] xNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    double[] yNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    double[] zNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    int J = (xNodes.Length - 1)*(yNodes.Length - 1)*(zNodes.Length - 1);
    
    string GridName = string.Format(wmg.CurrentProject + "_J" + J);
    
    grids[cnt] = wmg.Grids.SingleOrDefault(grd => grd.Name.Contains(string.Format("_J"+ J))); // check if an appropriate grid is already present in the database
    if(grids[cnt] == null){
        Console.WriteLine("Creating grid with " + J + " cells.");
        
        GridCommons g;
        g      = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
        g.Name = GridName;
        
        g.DefineEdgeTags(delegate (double[] X) {
            if (Math.Abs(X[0] - (-1)) <= 1.0e-8)
                return "wall_left";
            if (Math.Abs(X[0] - (+1)) <= 1.0e-8)
                return "wall_right";
            if (Math.Abs(X[1] - (-1)) <= 1.0e-8)
                return "wall_front";
            if (Math.Abs(X[1] - (+1)) <= 1.0e-8)
                return "wall_back";
            if (Math.Abs(X[2] - (-1)) <= 1.0e-8)
                return "wall_top";
            if (Math.Abs(X[2] - (+1)) <= 1.0e-8)
                return "wall_bottom";
            throw new ArgumentException("unknown wall");
        });
      
        g = wmg.SaveGrid(g);  
        grids[cnt] = g;
    } else {
        Console.WriteLine("Found Grid: " + grids[cnt]);
        if(grids[cnt].NumberOfCells != J)
            throw new Exception("J mismatch");
        
        if(grids[cnt].SpatialDimension != 3)
            throw new Exception("D mismatch");
    }
}

Opening existing database 'C:\BoSSStests\LinslvPerf_XdgStokes'.
Found Grid: { Guid = f2bd9c43-98d0-46d3-9a5f-26e0f749835e; Name = LinslvPerf_XdgStokes_J512; Cell Count = 512; Dim = 3 }
Found Grid: { Guid = fe686703-5eeb-48e4-a56d-cceffa806d9f; Name = LinslvPerf_XdgStokes_J4096; Cell Count = 4096; Dim = 3 }
Found Grid: { Guid = dfcd0a72-8e64-4edc-ba3d-ab7ab0181457; Name = LinslvPerf_XdgStokes_J13824; Cell Count = 13824; Dim = 3 }
Found Grid: { Guid = 68bcb2a5-770d-4fb8-9cfe-d43be11a8ed5; Name = LinslvPerf_XdgStokes_J32768; Cell Count = 32768; Dim = 3 }
Found Grid: { Guid = 1ea792c9-0abd-4137-b360-496388212de5; Name = LinslvPerf_XdgStokes_J110592; Cell Count = 110592; Dim = 3 }
Found Grid: { Guid = 70d7ea35-3f7e-44e4-affc-80f3e7d583bd; Name = LinslvPerf_XdgStokes_J262144; Cell Count = 262144; Dim = 3 }


In [7]:
grids

#0: { Guid = f2bd9c43-98d0-46d3-9a5f-26e0f749835e; Name = LinslvPerf_XdgStokes_J512; Cell Count = 512; Dim = 3 }
#1: { Guid = fe686703-5eeb-48e4-a56d-cceffa806d9f; Name = LinslvPerf_XdgStokes_J4096; Cell Count = 4096; Dim = 3 }
#2: { Guid = dfcd0a72-8e64-4edc-ba3d-ab7ab0181457; Name = LinslvPerf_XdgStokes_J13824; Cell Count = 13824; Dim = 3 }
#3: { Guid = 68bcb2a5-770d-4fb8-9cfe-d43be11a8ed5; Name = LinslvPerf_XdgStokes_J32768; Cell Count = 32768; Dim = 3 }
#4: { Guid = 1ea792c9-0abd-4137-b360-496388212de5; Name = LinslvPerf_XdgStokes_J110592; Cell Count = 110592; Dim = 3 }
#5: { Guid = 70d7ea35-3f7e-44e4-affc-80f3e7d583bd; Name = LinslvPerf_XdgStokes_J262144; Cell Count = 262144; Dim = 3 }


## Setup Control Object for a Solver Run

In [8]:
// - - - - - - - - - - - - - - - - - - -
// Initial Values & Boundary conditions
// - - - - - - - - - - - - - - - - - - -

In [9]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.Timestepping;
using BoSSS.Solution.XdgTimestepping ;

### Setup of Parameter Study

Polynomial degrees to test:

In [10]:
int[] PolyDegS = new int[] {2, 3 ,5};

Solvers which we want to instrument:

In [11]:
// Solvers which we want to instrument:
LinearSolverCode[] solver_nameS = new LinearSolverCode[] {
    LinearSolverCode.classic_pardiso,
    LinearSolverCode.exp_gmres_levelpmg,
    LinearSolverCode.exp_Kcycle_schwarz,
}; 

Maximum Dof for one calculation (we skip fine grids for higher polynomial orders):

In [12]:
int GetMaxAllowedDOF(LinearSolverCode code) {
    switch(code) {
        case LinearSolverCode.classic_pardiso:
        case LinearSolverCode.classic_mumps:
        return 1100000; // 1.1 Million for iterative solvers at maximum
    
        default: 
        return 5000000; // Up to 5 Million for iterative solvers
    }
}

Loop over all combinations of parameters and define a control object for each combo:

In [13]:
using BoSSS.Solution.XNSECommon;
using BoSSS.Foundation.XDG;

In [14]:
List<XNSE_Control> controls = new List<XNSE_Control>();
controls.Clear();
foreach(LinearSolverCode solver in solver_nameS) {
foreach(int k in PolyDegS) {
foreach(IGridInfo grd in grids) {

    int Np_V = Utils.Np(k);
    int Np_p = Utils.Np(k-1);
    int J    = grd.NumberOfCells;
    int DOF  = Np_V*3 + Np_p;
    if(J*DOF >  GetMaxAllowedDOF(solver))
        continue;
    
    // Control Instance, grid, DG degree, etc.
    // =======================================
    
    XNSE_Control C = new XNSE_Control();
    controls.Add(C);
       
    string caseName = string.Format("XdgStokes-J{0}_p{1}_{2}", J, k, solver);
    Console.WriteLine("setting up: " + caseName);
    C.SessionName        = caseName;
    
    C.SetGrid(grd);
    C.savetodb = true;
    C.SetDGdegree(k);
    
    // Phys. Parameters
    // ================
    
    // Species A: Water; Species B: Air
    C.PhysicalParameters.rho_A             = 1e-3; //     kg / cm³
    C.PhysicalParameters.rho_B             = 1.2e-6; //   kg / cm³
    C.PhysicalParameters.mu_A              = 1e-5; //      kg / cm / sec
    C.PhysicalParameters.mu_B              = 17.1e-8; //   kg / cm / sec
    C.PhysicalParameters.Sigma             = 72.75e-3; // kg / sec²   
    C.PhysicalParameters.IncludeConvection = false;
    C.PhysicalParameters.Material          = true;
    
    // Dont know
    // ============
    
    double r     = 0.5;
    double nonsp = 0.5;

    C.AddInitialValue("Phi", new Formula($"X => (X[0]/{r*nonsp}).Pow2() + (X[1]/{r}).Pow2() + (X[2]/{r}).Pow2() - 1", false));
    
    C.LSContiProjectionMethod = BoSSS.Solution.LevelSetTools.ContinuityProjectionOption.None;
    //C.CutCellQuadratureType   = BoSSS.Foundation.XDG.XQuadFactoryHelper.MomentFittingVariants.Saye;
    C.ComputeEnergyProperties = false;

    
    // Solver Stuff
    // ============
    
    C.VelocityBlockPrecondMode         = MultigridOperator.Mode.SymPart_DiagBlockEquilib;
    
    C.LinearSolver.SolverCode           = solver;
    C.LinearSolver.ConvergenceCriterion = 1e-8;
    C.LevelSet_ConvergenceCriterion     = 1e-6;

    //C.Option_LevelSetEvolution                          = LevelSetEvolution.FastMarching;
    C.AdvancedDiscretizationOptions.SST_isotropicMode   = SurfaceStressTensor_IsotropicMode.Curvature_Projected;
    //C.AdvancedDiscretizationOptions.ViscosityMode       = ViscosityMode.Standard;
    C.AdvancedDiscretizationOptions.FilterConfiguration = CurvatureAlgorithms.FilterConfiguration.NoFilter;

    // Timestepping / Instationary
    // ===========================

    C.TimesteppingMode             = AppControl._TimesteppingMode.Steady;
}
}
}

setting up: XdgStokes-J512_p2_classic_pardiso
setting up: XdgStokes-J4096_p2_classic_pardiso
setting up: XdgStokes-J13824_p2_classic_pardiso
setting up: XdgStokes-J512_p3_classic_pardiso
setting up: XdgStokes-J4096_p3_classic_pardiso
setting up: XdgStokes-J13824_p3_classic_pardiso
setting up: XdgStokes-J512_p5_classic_pardiso
setting up: XdgStokes-J4096_p5_classic_pardiso
setting up: XdgStokes-J512_p2_exp_gmres_levelpmg
setting up: XdgStokes-J4096_p2_exp_gmres_levelpmg
setting up: XdgStokes-J13824_p2_exp_gmres_levelpmg
setting up: XdgStokes-J32768_p2_exp_gmres_levelpmg
setting up: XdgStokes-J110592_p2_exp_gmres_levelpmg
setting up: XdgStokes-J512_p3_exp_gmres_levelpmg
setting up: XdgStokes-J4096_p3_exp_gmres_levelpmg
setting up: XdgStokes-J13824_p3_exp_gmres_levelpmg
setting up: XdgStokes-J32768_p3_exp_gmres_levelpmg
setting up: XdgStokes-J512_p5_exp_gmres_levelpmg
setting up: XdgStokes-J4096_p5_exp_gmres_levelpmg
setting up: XdgStokes-J13824_p5_exp_gmres_levelpmg
setting up: XdgStokes

Total number of simulations:

In [15]:
controls.Count

32

In [44]:
(new XNSE_Control()).LinearSolver

verbose,SolverCode,MaxKrylovDim,MaxSolverIterations,MinSolverIterations,ConvergenceCriterion,NoOfMultigridLevels,SolverMode,exp_localPrec_muA,exp_localPrec_Min_dt,TargetBlockSize,pMaxOfCoarseSolver,UsePMGonly
False,classic_mumps,100,2000,4,1E-10,1,Solve,1,0,10000,1,False


In [16]:
// Assert that the location of the fluid phases is as desired:
NUnit.Framework.Assert.Negative(controls[0].InitialValues["Phi"].Evaluate(new double[]{0,0.0,0}, 0.0), "phase A (negative) must be inside");
NUnit.Framework.Assert.Positive(controls[0].InitialValues["Phi"].Evaluate(new double[]{0,4.0,0}, 0.0), "phase B (positive) must be inside");

## Launch Jobs

In [17]:
foreach(var ctrl in controls) {
    Console.WriteLine(" Submitting: " + ctrl.SessionName); 
    ctrl.RunBatch(myBatch);
}

 Submitting: XdgStokes-J512_p2_classic_pardiso
Info: Found successful session "LinslvPerf_XdgStokes	XdgStokes-J512_p2_classic_pardiso	11/11/2021 10:17:18	590d8589..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: XdgStokes-J4096_p2_classic_pardiso
Info: Found successful session "LinslvPerf_XdgStokes	XdgStokes-J4096_p2_classic_pardiso	11/11/2021 10:17:31	1daae1c4..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: XdgStokes-J13824_p2_classic_pardiso
Info: Found successful session "LinslvPerf_XdgStokes	XdgStokes-J13824_p2_classic_pardiso	11/11/2021 10:17:45	fd18b34a..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: XdgStokes-J512_p3_classic_pardiso
Info: Found successful session "LinslvPerf_XdgStokes	XdgStokes-J512_p3_classic_pardiso	11/11/2021 10:17:56	0500cbb1..." --

Hint: want to re-activate the job.
No submission, because job status is: FailedOrCanceled
 Submitting: XdgStokes-J13824_p5_exp_Kcycle_schwarz
 ------------ MSHPC FailedOrCanceled; original Failed
Note: Job was deployed (1) number of times, all failed.
Hint: want to re-activate the job.
No submission, because job status is: FailedOrCanceled


### Wait for Completion and Check Job Status

In [18]:
wmg.BlockUntilAllJobsTerminate(3600*24*2); // wait at maximum two days for the jobs to finish

All jobs finished.


In [19]:
wmg.AllJobs

#0: XdgStokes-J512_p2_classic_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)	XdgStokes-J512_p2_classic_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#1: XdgStokes-J4096_p2_classic_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)	XdgStokes-J4096_p2_classic_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#2: XdgStokes-J13824_p2_classic_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)	XdgStokes-J13824_p2_classic_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#3: XdgStokes-J512_p3_classic_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)	XdgStokes-J512_p3_classic_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\Val

In [36]:
wmg.Sessions

#0: LinslvPerf_XdgStokes	XdgStokes-J4096_p5_classic_pardiso*	11/12/2021 05:39:35	4319da29...
#1: LinslvPerf_XdgStokes	XdgStokes-J110592_p2_exp_gmres_levelpmg	11/11/2021 18:12:42	9f289d1e...
#2: LinslvPerf_XdgStokes	XdgStokes-J32768_p2_exp_Kcycle_schwarz	11/11/2021 10:22:45	0e5f41a4...
#3: LinslvPerf_XdgStokes	XdgStokes-J32768_p3_exp_gmres_levelpmg	11/11/2021 10:20:52	4714a39b...
#4: LinslvPerf_XdgStokes	XdgStokes-J4096_p5_exp_gmres_levelpmg	11/11/2021 10:21:24	81c82c15...
#5: LinslvPerf_XdgStokes	XdgStokes-J13824_p3_exp_gmres_levelpmg	11/11/2021 10:20:38	630aead5...
#6: LinslvPerf_XdgStokes	XdgStokes-J32768_p2_exp_gmres_levelpmg	11/11/2021 10:19:44	de764dde...
#7: LinslvPerf_XdgStokes	XdgStokes-J13824_p2_classic_pardiso	11/11/2021 10:17:45	fd18b34a...
#8: LinslvPerf_XdgStokes	XdgStokes-J4096_p3_exp_Kcycle_schwarz	11/11/2021 10:23:38	13191495...
#9: LinslvPerf_XdgStokes	XdgStokes-J13824_p2_exp_Kcycle_schwarz	11/11/2021 10:22:27	54d85875...
#10: LinslvPerf_XdgStokes	XdgStokes-J4096_p3_ex

In [46]:
//wmg.Sessions[3].Export().WithSupersampling(2).Do()

In [21]:
var NoSuccess = wmg.AllJobs.Values.Where(job => job.Status != JobStatus.FinishedSuccessful);
NoSuccess

#0: XdgStokes-J13824_p3_classic_pardiso: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#1: XdgStokes-J4096_p5_classic_pardiso: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#2: XdgStokes-J13824_p2_exp_gmres_levelpmg: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#3: XdgStokes-J13824_p5_exp_gmres_levelpmg: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#4: XdgStokes-J110592_p2_exp_Kcycle_schwarz: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#5: XdgStokes-J13824_p3_exp_Kcycle_schwarz: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#6: XdgStokes-J32768_p3_exp_Kcycle_schwarz: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#7: XdgStokes-J4096_p5_exp_Kcycle_schwarz: FailedOrCanceled (MS HPC client  MSHPC-AllNode

In [27]:
// In the case of some failed job, print the directory name for further inspection:
foreach(var fail in NoSuccess)
    Console.WriteLine(fail + ":  @" + fail.LatestDeployment.DeploymentDirectory.FullName);
    //Console.WriteLine(fail.LatestDeployment);

XdgStokes-J13824_p3_classic_pardiso: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests):  @\\fdygitrunner\ValidationTests\LinslvPerf_XdgStokes-XNSE_Solver2021Nov11_101805
XdgStokes-J4096_p5_classic_pardiso: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests):  @\\fdygitrunner\ValidationTests\LinslvPerf_XdgStokes-XNSE_Solver2021Nov11_101828
XdgStokes-J13824_p2_exp_gmres_levelpmg: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests):  @\\fdygitrunner\ValidationTests\LinslvPerf_XdgStokes-XNSE_Solver2021Nov11_101911
XdgStokes-J13824_p5_exp_gmres_levelpmg: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests):  @\\fdygitrunner\ValidationTests\LinslvPerf_XdgStokes-XNSE_Solver2021Nov11_102122
XdgStokes-J110592_p2_exp_Kcycle_schwarz: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests):  @\\fdygitrunner\ValidationTests\Linsl

In [23]:
var FailedSessions = wmg.Sessions.Where(Si => (Si.SuccessfulTermination == false));
FailedSessions

#0: LinslvPerf_XdgStokes	XdgStokes-J110592_p2_exp_Kcycle_schwarz*	11/11/2021 10:23:00	7519691b...
#1: LinslvPerf_XdgStokes	XdgStokes-J32768_p3_exp_Kcycle_schwarz*	11/11/2021 10:24:14	314c737a...
#2: LinslvPerf_XdgStokes	XdgStokes-J13824_p5_exp_Kcycle_schwarz*	11/11/2021 10:25:06	aad97ce0...
#3: LinslvPerf_XdgStokes	XdgStokes-J4096_p5_exp_Kcycle_schwarz*	11/11/2021 10:24:47	7593ffc2...
#4: LinslvPerf_XdgStokes	XdgStokes-J13824_p3_exp_Kcycle_schwarz*	11/11/2021 10:23:58	e0cbff63...
#5: LinslvPerf_XdgStokes	XdgStokes-J110592_p2_exp_gmres_levelpmg*	11/11/2021 10:19:57	aaa45e54...
#6: LinslvPerf_XdgStokes	XdgStokes-J13824_p5_exp_gmres_levelpmg*	11/11/2021 10:21:38	0c5867d5...
#7: LinslvPerf_XdgStokes	XdgStokes-J13824_p2_exp_gmres_levelpmg*	11/11/2021 10:19:32	464e8d24...
#8: LinslvPerf_XdgStokes	XdgStokes-J13824_p3_classic_pardiso*	11/11/2021 10:18:23	8d4759a7...
#9: LinslvPerf_XdgStokes	XdgStokes-J4096_p5_classic_pardiso*	11/11/2021 10:18:44	d025fc40...


In [24]:
//foreach(var si in FailedSessions)
//   si.Delete(true);

In [25]:
NUnit.Framework.Assert.Zero(NoSuccess.Count(), "Some Jobs Failed");

Unhandled exception: NUnit.Framework.AssertionException:   Some Jobs Failed
  Expected: 0
  But was:  9

   at NUnit.Framework.Assert.ReportFailure(String message) in /_/src/NUnitFramework/framework/Assert.cs:line 395
   at NUnit.Framework.Assert.ReportFailure(ConstraintResult result, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.cs:line 383
   at NUnit.Framework.Assert.That[TActual](TActual actual, IResolveConstraint expression, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.That.cs:line 229
   at NUnit.Framework.Assert.Zero(Int32 actual, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.Conditions.cs:line 484
   at Submission#27.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchE

In [26]:
NUnit.Framework.Assert.Zero(FailedSessions.Count(), "Some Sessions did not terminate successfully.");

Unhandled exception: NUnit.Framework.AssertionException:   Some Sessions did not terminate successfully.
  Expected: 0
  But was:  10

   at NUnit.Framework.Assert.ReportFailure(String message) in /_/src/NUnitFramework/framework/Assert.cs:line 395
   at NUnit.Framework.Assert.ReportFailure(ConstraintResult result, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.cs:line 383
   at NUnit.Framework.Assert.That[TActual](TActual actual, IResolveConstraint expression, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.That.cs:line 229
   at NUnit.Framework.Assert.Zero(Int32 actual, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.Conditions.cs:line 484
   at Submission#28.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exc